## Film Content Insights



## Overview

This project analyzes current trends in the film industry by examining the performance of different genres at the box office. By investigating various datasets, including Box Office Mojo, IMDB, Rotten Tomatoes, TheMovieDB, and The Numbers, we aim to identify what types of films are currently most successful. This analysis will help in forecasting which film genres hold the most promise for profitability and audience engagement, thus guiding strategic decisions related to film production, marketing, and distribution.

## Business Problem

The film industry is highly competitive and continuously evolving, with varying audience preferences and technological advancements shaping market dynamics. Understanding which film genres are performing well at the box office can enable the newly established movie studio to allocate resources effectively, maximize returns, and expand its market presence. By leveraging detailed box office data, the studio can make informed decisions about which types of films to produce, potentially leading to increased profitability and audience acclaim.

<img src="https://wallpapercave.com/wp/wp8021237.jpg" width="300" alt="Descriptive Text">


## Data Understanding

The datasets include a mix of structured data from well-known film databases, covering extensive details about film genres, box office earnings, ratings, and audience feedback across several years. Each film is identified uniquely, allowing for precise tracking of its performance from release to international earnings. This comprehensive data enables an in-depth analysis of market trends, audience preferences, and financial outcomes associated with different film types.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os




# Setting visualisation styles
sns.set(style="whitegrid")


# Load TSV files
rt_movie_info = pd.read_csv(r'C:\Users\neali\Documents\Flatiron\2\dsc-phase-2-project\zippedData\rt.movie_info.tsv.gz', delimiter='\t', compression='gzip')
# Load TSV files using latin-1 encoding
rt_reviews = pd.read_csv(
    r'C:\Users\neali\Documents\Flatiron\2\dsc-phase-2-project\zippedData\rt.reviews.tsv.gz',
    delimiter='\t',
    compression='gzip',
    encoding='latin-1'
)


# Load CSV files
bom_movie_gross = pd.read_csv(r'C:\Users\neali\Documents\Flatiron\2\dsc-phase-2-project\zippedData\bom.movie_gross.csv.gz', compression='gzip')
tmdb_movies = pd.read_csv(r'C:\Users\neali\Documents\Flatiron\2\dsc-phase-2-project\zippedData\tmdb.movies.csv.gz', compression='gzip')
tn_movie_budgets = pd.read_csv(r'C:\Users\neali\Documents\Flatiron\2\dsc-phase-2-project\zippedData\tn.movie_budgets.csv.gz', compression='gzip')


# Path to the zip file
zip_file_path = r'C:\Users\neali\Documents\Flatiron\2\phase-2-project-v3\dsc-phase-2-project-v3\zippedData\im.db.zip'
# Directory where the db file will be extracted
extraction_path = r'C:\Users\neali\Documents\Flatiron\2\phase-2-project-v3\dsc-phase-2-project-v3\zippedData'

# Unzip the database file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extraction_path)

# Assuming the database file is named 'im.db' and is the only file in the zip
db_file_path = os.path.join(extraction_path, 'im.db')

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Now you can perform database operations
# Example: Listing all tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table)





# Querying data from SQLite database
query = """
SELECT *
FROM movie_basics
JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id;
"""
movie_data = pd.read_sql_query(query, conn)

## Data Processing with Pandas and SQLite

Next, we will perform a series of SQL and Pandas operations to retrieve and process data from the IMDB movie database, focusing on movie ratings and related attributes. The top 20 unique ratings are displayed, showcasing key information about each movie, including its title, director, character involvement, average rating, and total votes. This data is particularly useful for analyzing viewer preferences and the impact of directorial roles on movie ratings.

In [ ]:
#Combine Data
sql_query2 = """

SELECT 
    mb.primary_title, 
    p.primary_name, 
    mb.genres,
    mr.averagerating,
    mr.numvotes
FROM 
    movie_basics mb
JOIN 
    directors d ON mb.movie_id = d.movie_id
JOIN 
    persons p ON d.person_id = p.person_id
JOIN 
    principals pr ON p.person_id = pr.person_id
JOIN 
    movie_ratings mr ON mb.movie_id = mr.movie_id
GROUP BY 
    mb.movie_id, p.primary_name
ORDER BY 
    mb.movie_id ASC;
"""



# Assuming 'conn' is already your active connection to the SQLite database
result = pd.read_sql_query(sql_query2, conn)
result.head()



# Drop duplicates based on 'primary_title' and 'averagerating'
unique_ratings = result.drop_duplicates(subset=['primary_title', 'averagerating'])

# Sort the DataFrame by 'numvotes' in descending order
sorted_unique_ratings = unique_ratings.sort_values(by='numvotes', ascending=False)

# Select the top 20 unique ratings
top_20_unique_ratings = sorted_unique_ratings.head(20)

# Display the top 20 unique ratings
top_20_unique_ratings









Next, we will read and analyze the CSV and TSV files in order to then merge and perform analysis with the queried database

In [110]:
# List columns of each DataFrame
dataframes = {
    'RT Movie Info': rt_movie_info,
    'RT Reviews': rt_reviews,
    'BOM Movie Gross': bom_movie_gross,
    'TMDB Movies': tmdb_movies,
    'TN Movie Budgets': tn_movie_budgets
}

# Print columns for each DataFrame
for name, df in dataframes.items():
    print(f"Columns in {name}:")
    print(df.columns.tolist())
    print()


Columns in RT Movie Info:
['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime', 'studio']

Columns in RT Reviews:
['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher', 'date']

Columns in BOM Movie Gross:
['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']

Columns in TMDB Movies:
['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count']

Columns in TN Movie Budgets:
['id', 'release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']



## Analysis 1: Find ROI for highest grossing movies based on Genre

The next step is to merge dataframes in order to see highest grossing and viewed films, then create a new column calculating the Return on Investment (ROI) and finally, sort and group by Genre.

In [148]:
# Rename 'title' to 'primary_title' in bom_movie_gross for consistency
bom_movie_gross = bom_movie_gross.rename(columns={'title': 'primary_title'})

# Merge unique_ratings with bom_movie_gross
merged_df = pd.merge(unique_ratings, bom_movie_gross[['primary_title']],
                     on='primary_title', how='left')

# Merge the result with tn_movie_budgets
final_merged_df = pd.merge(merged_df, tn_movie_budgets[['primary_title', 'production_budget', 'domestic_gross', 'worldwide_gross']],
                           on='primary_title', how='left')


# Set display options to suppress scientific notation
pd.set_option('display.float_format', '{:.1f}'.format)


# Convert currency columns from string to float
final_merged_df['production_budget'] = final_merged_df['production_budget'].replace('[\$,]', '', regex=True).astype(float)
final_merged_df['domestic_gross'] = final_merged_df['domestic_gross'].replace('[\$,]', '', regex=True).astype(float)
final_merged_df['worldwide_gross'] = final_merged_df['worldwide_gross'].replace('[\$,]', '', regex=True).astype(float)

# Calculate ROI Domestic
final_merged_df['ROI Domestic'] = (final_merged_df['domestic_gross'] / final_merged_df['production_budget']) * 100

# Calculate ROI Worldwide
final_merged_df['ROI Worldwide'] = (final_merged_df['worldwide_gross'] / final_merged_df['production_budget']) * 100
# Optional: Remove duplicates based on specific columns
final_merged_df = final_merged_df.drop_duplicates(subset=['primary_title', 'primary_name'])
# Optional: Remove rows with NaN values in specific columns
final_merged_df = final_merged_df.dropna(subset=['production_budget', 'domestic_gross', 'worldwide_gross'])

# Display the first few rows of the final DataFrame
final2 = final_merged_df.sort_values(by='numvotes', ascending=False)

final_gross = final2.head(30)

# Continue analysis and filter for genre by aggregating Budget, gross, and ROIs per genre

# Define the aggregation methods for each column
#aggregations = {
    #'primary_title': 'count',  # Count the number of movies per genre
    #'production_budget': ['sum', 'mean'],  # Sum and average budget per genre
    #'domestic_gross': ['sum', 'mean'],  # Sum and average domestic gross per genre
    #'worldwide_gross': ['sum', 'mean'],  # Sum and average worldwide gross per genre
    #'ROI Domestic': 'mean',  # Average ROI domestically
    #'ROI Worldwide': 'mean'  # Average ROI worldwide
#}

# Explode genres into separate rows for movies that list multiple genres
#final_merged_df['genres'] = final_merged_df['genres'].str.split(',')
#exploded_df = final_merged_df.explode('genres')

# Now group by the single 'genres' entries and aggregate
#genre_grouped_exploded = exploded_df.groupby('genres').agg(aggregations)

# Simplify column names as before
#genre_grouped_exploded.columns = ['_'.join(col).strip() for col in genre_grouped_exploded.columns.values]

# Print the resulting DataFrame
#genre_grouped_exploded



# Assuming final_merged_df is your DataFrame

# Handle any potential NaN values in 'genres'
final_merged_df['genres'] = final_merged_df['genres'].fillna('Unknown')

# Split genres into lists, ensuring each entry is non-null and a string
final_merged_df['genres'] = final_merged_df['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else ['Unknown'])

# Create a column for the number of genres per movie
final_merged_df['genre_count'] = final_merged_df['genres'].apply(len)

# Now explode the genres into separate rows for detailed analysis
exploded_df = final_merged_df.explode('genres')

# Apply weights to monetary fields directly in exploded_df using genre_count from the same df
exploded_df['production_budget_weighted'] = exploded_df['production_budget'] / exploded_df['genre_count']
exploded_df['domestic_gross_weighted'] = exploded_df['domestic_gross'] / exploded_df['genre_count']
exploded_df['worldwide_gross_weighted'] = exploded_df['worldwide_gross'] / exploded_df['genre_count']

# Aggregate the data
aggregated_df = exploded_df.groupby('genres').agg({
    'production_budget_weighted': ['sum', 'mean'],
    'domestic_gross_weighted': ['sum', 'mean'],
    'worldwide_gross_weighted': ['sum', 'mean'],
    'ROI Domestic': 'mean',  # Assuming ROI calculations are already weighted
    'ROI Worldwide': 'mean'
}).reset_index()

# Simplify column names
aggregated_df.columns = ['_'.join(col).strip() for col in aggregated_df.columns.values]

# Display the results
aggregated_df




,genres_,production_budget_weighted_sum,production_budget_weighted_mean,domestic_gross_weighted_sum,domestic_gross_weighted_mean,worldwide_gross_weighted_sum,worldwide_gross_weighted_mean,ROI Domestic_mean,ROI Worldwide_mean
0,Action,14013393333.3,23473020.7,15190804641.8,25445233.9,41035406837.0,68736024.9,161.2,350.2
1,Adventure,13812221368.7,32271545.3,16364271606.2,38234279.5,46270787093.0,108109315.6,116.0,290.7
2,Animation,4053750000.0,32430000.0,5400346029.2,43202768.2,14854483610.2,118835868.9,129.0,340.2
3,Biography,1730565741.7,9060553.6,2399286250.5,12561708.1,4982196485.2,26084798.4,206.8,446.7
4,Comedy,10806895006.3,14525396.5,15188468728.5,20414608.5,32478233652.3,43653539.9,178.8,332.7
5,Crime,3557023624.3,10133970.4,4074488310.5,11608228.8,8846965801.0,25205030.8,129.0,231.6
6,Documentary,3452475535.3,18075788.1,4113555069.7,21536937.5,8311449066.7,43515440.1,230.1,415.5
7,Drama,16103875611.3,11469996.9,19690941722.2,14024887.3,40547407241.7,28879919.7,175.4,320.9
8,Family,2569245833.3,19031450.6,3047327636.0,22572797.3,7286082403.2,53970980.8,177.2,290.6
9,Fantasy,4053416666.7,25333854.2,4153611786.3,25960073.7,12048767066.5,75304794.2,149.9,334.5
